In [1]:
import pandas as pd
import folium
import time
import re

In [6]:
pwd

'/Users/yinanli/Documents/GitHub/generic-real-estate-consulting-project-group-7/notebooks/ors_nearest_external_by_postcode'

In [7]:
df_all_centroid = pd.read_csv('../../data/raw/external-data/postcode_centroid.csv')

In [8]:
df_care_facility = pd.read_csv("../../data/raw/external-data/care_facility.csv")

In [4]:
# extract latitude and longitude from the geometry column using regular expression
loc = {"LATITUDE":[], "LONGITUDE":[]}
for i in range(0, len(df_care_facility)):
    lon = float(re.findall(r"\d+\.?\d*", df_care_facility['geometry'][i])[0])
    lat = float(re.findall(r"\d+\.?\d*", df_care_facility['geometry'][i])[1])
    loc["LATITUDE"].append(-lat)
    loc["LONGITUDE"].append(lon)

In [5]:
# create two coloumns for latitude and longitude
df_care_facility['LATITUDE'] = loc["LATITUDE"]
df_care_facility['LONGITUDE'] = loc["LONGITUDE"]

In [6]:
import openrouteservice as ors
client = ors.Client(key='5b3ce3597851110001cf624856044b6ee25648c9918b1adf78eedc57')

In [7]:
def nearest(df_all_property, df_facility, index):
    i = index
    
    # get the postcode find all facilities in this postcode
    postcode = df_all_property['POSTCODE'][i]
    facility = df_facility.loc[df_facility['postcode'] == postcode].to_dict()

    # get the longitude and latitude of the these facilities
    lat = list(facility["LATITUDE"].values())
    lon = list(facility["LONGITUDE"].values())
   
    # if there is no facility in this suburb, early return none
    if len(lat) == 0:
        return (postcode, 'none', 'none', 'none', 'none')
 
    # create a list of location, put the centroid of this suburb as the first element
    loc = [list(eval(df_all_property['centroid'][i]))]
        
        
    # append the loctations of the facilities in this suburb to the list of location
    for j in range(0, len(lat)):
        loc.append([lon[j], lat[j]])
        

    # get the driving distance and duration between the suburb centroid and each facility
    matrix = client.distance_matrix(
    locations = loc,
    sources = list(range(1, len(loc))),
    destinations = [0],
    metrics=['distance', 'duration'],
    )
    
    #get the shortest duration among all facilities
    nearest_distance = min(matrix['distances'])[0]
    nearest_duration = min(matrix['durations'])[0]                                                    
     
    # get the name and the longitude and latitude of that facility with shortest duration
    index = matrix['durations'].index(min(matrix['durations']))
    nearest_name = list(facility['NAME_LABEL'].values())[index]
    nearest_latlon = loc[index + 1] 

    return (postcode, nearest_name, nearest_distance, nearest_duration, nearest_latlon)


In [8]:
# create a dictionary of care_facility to store the information
care_facility = {"postcode":[], "nearest_latlon":[], "nearest_name":[],
                 "nearest_duration":[], "nearest_distance":[]}

In [9]:
for i in range(0, len(df_all_centroid)):   
    
    # sleep 10 seconds each 20 postcode to avoid timeout error
    if isinstance(i % 20, int) and (i % 20 == 0):
        time.sleep(10) 
       
    # get the information of the nearest facility (shortest driving duration) in each postcode
    # using the nearest fuction
    result = nearest(df_all_centroid, df_care_facility, i)
    
    # append the result to the dictionary
    care_facility["nearest_name"].append(result[1])
    care_facility["nearest_distance"].append(result[2])
    care_facility["nearest_duration"].append(result[3])
    care_facility["postcode"].append(result[0])
    care_facility["nearest_latlon"].append(result[4])

/Users/zongchaoxie/opt/anaconda3/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/zongchaoxie/opt/anaconda3/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/zongchaoxie/opt/anaconda3/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Users/zongchaoxie/opt/anaconda3/lib/python3.9/site-packages/openrouteservice/client.py:211: UserWarning: Rate limit exceeded. Retrying for the 4th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
/Use

In [10]:
# build a dataframe using the dictionary
postcode_nearest_care_facility = pd.DataFrame(care_facility)

In [12]:
# save the data
postcode_nearest_care_facility.to_csv("../data/raw/ors-data/care_facility_by_postcode.csv")